Demo

# Go to cell 20 to see submission

In [1]:
# Auto Encoding (for NLU purpose)

In [2]:
# Hugging Face

In [3]:
 # installing required libraries
!pip install transformers
!pip install sentencepiece
!pip install contractions

In [4]:
# importing required libraries
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from tqdm import tqdm, trange
from ast import literal_eval
import json
import nltk
from nltk.tokenize import LineTokenizer, sent_tokenize
nltk.download('punkt')
import re
import contractions
from google.colab import drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# mounting google drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [6]:
# This function processes the text so that it can be used to produce meaningful embeddings
def processText(text):

    # Using the line tokenizer to split using new line character (\n) and discarding the blank lines
    if text is not None:

        lines = LineTokenizer(blanklines = 'discard').tokenize(text)

        sentences = []

        # For each line found above we are doing sentence tokenization
        for line in lines:
            currentSentences = sent_tokenize(line)
            # sentences.extend([sentence for sentence in currentSentences])
            for sentence in currentSentences:
                currentText = str(sentence.encode("ascii", "ignore").decode().strip())
                sentences.extend([currentText])
                text = '. '.join([processSentence(sentence) for sentence in sentences])
    return text


# This function processes the sentence so that it can be used to produce meaningful embeddings
def processSentence(text):
    # Removing HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Removing new line characters
    text = text.replace('\\n', '')
    # Removing urls
    # text = re.sub('[^ ]+\.[^ ]+', ' ', text)
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''," ",text)
    # Removing email addresses
    text = re.sub(r'''\S*@\S*\s?''', " ", text)
    # Expanding the contractions
    text = contractions.fix(text)
    return text

In [7]:
df = pd.read_csv("/content/drive/My Drive/training.1600000.processed.noemoticon.csv", encoding='mac_roman', header = None)
df = df.sample(1600)
df[0] = df[0].apply(lambda x: 0 if x == 0 else 1)
df

,0,1,2,3,4,5
7854,0,1469974463,Tue Apr 07 07:58:39 PDT 2009,NO_QUERY,sassafrasstic,my twitter is so much more boring without @joh...
12930,1,2191843587,Tue Jun 16 06:04:11 PDT 2009,NO_QUERY,dcynaira,Dreams are nothing but nightmares if you don't...
18501,1,2193188165,Tue Jun 16 08:06:57 PDT 2009,NO_QUERY,SallytheShizzle,@wimjimjam hahaha! Well all I can say is... YO...
1465,0,1468160042,Tue Apr 07 00:02:35 PDT 2009,NO_QUERY,jooyoung_yeu,oh god one of the teachers here gave me a rott...
5864,0,1469218042,Tue Apr 07 05:38:52 PDT 2009,NO_QUERY,uniquest1,why do i only think of good april fools jokes ...
...,...,...,...,...,...,...
17174,1,2192860072,Tue Jun 16 07:39:21 PDT 2009,NO_QUERY,ambersturgis,Finished that paper with an hour and a half to...
8840,0,1548496797,Fri Apr 17 21:06:59 PDT 2009,NO_QUERY,ScarlettDane,"@Chels_V I am good, just about to leave for Fo..."
8314,0,1470150659,Tue Apr 07 08:29:57 PDT 2009,NO_QUERY,AlanLB,"@tgadget Thanks for your tweet, but I am Hop..."
3840,0,1468686594,Tue Apr 07 03:09:58 PDT 2009,NO_QUERY,DivasMistress,@nikkiwoods Exactamundo!!! For some reason I t...


In [8]:
comments = list(df[5])
labels = list(df[0])

In [9]:
# tokenizing using RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
encodings = tokenizer.batch_encode_plus(comments, max_length = 512, pad_to_max_length = True)

print('tokenizer outputs: ', encodings.keys())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tokenizer outputs:  dict_keys(['input_ids', 'attention_mask'])


In [10]:
# getting tokenized and encoded sentences and attention masks
input_ids = encodings['input_ids']
attention_masks = encodings['attention_mask']

In [11]:
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks, random_state = 303, test_size = 500)

print("x_train", len(train_inputs))
print("x_vad", len(validation_inputs))
print("y_train", len(train_labels))
print("y_vad", len(validation_labels))

x_train 1100
x_vad 500
y_train 1100
y_vad 500


In [12]:
# Converting all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
train_masks = torch.tensor(attention_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [13]:
# Creating an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = 16)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler = validation_sampler, batch_size = 16)

In [14]:
# Load model, the pretrained model will include a single linear classification layer on top for classification.
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)
model.cuda()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [15]:
# setting custom optimization parameters.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0
    }
]

optimizer = AdamW(optimizer_grouped_parameters, lr = 2e-5, correct_bias = True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
# Train Model
torch.cuda.memory_summary(device = None, abbreviated = False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      | 488175 KiB | 488175 KiB | 488175 KiB |      0 B   |\n|       from large pool | 487680 KiB | 487680 KiB | 487680 KiB |      0 B   |\n|       from small pool |    495 KiB |    495 KiB |    495 KiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         | 488175 KiB | 488175 KiB | 488175 KiB |      0 B   |\n|       from large pool | 487680 KiB | 487680 KiB |

In [17]:
# Store our loss and accuracy for plotting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc = "Epoch"):

    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0 #running loss
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):

        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # print(b_input_ids)
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        # # Forward pass for multiclass classification
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # print("output type: ", type(outputs))
        loss = outputs[0].float()
        # print("loss type: ", type(loss.item()))
        logits = outputs[1].float()

        # Forward pass for multilabel classification
        # outputs = model(b_input_ids, attention_mask = b_input_mask)
        # logits = outputs[0]
        # loss_func = BCEWithLogitsLoss()
        # loss = loss_func(logits.view(-1,2), b_labels.type_as(logits).view(-1,2)) #convert labels to float for calculation
        # loss_func = BCELoss()
        # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
        # train_loss_set.append(loss.item())

        # Backward pass
        loss.backward()

        # Update parameters and take a step using the computed gradient
        optimizer.step()
        # scheduler.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

        print("Train loss: {}".format(tr_loss / nb_tr_steps))

###############################################################################

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Variables to gather full output
    logit_preds, true_labels, pred_labels, tokenized_texts = [],[],[],[]

    # Predict
    for i, batch in enumerate(validation_dataloader):

        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # print(b_input_ids)
        with torch.no_grad():

            # Forward pass
            outs = model(b_input_ids, attention_mask=b_input_mask)
            b_logit_pred = outs[0]
            pred_label = torch.sigmoid(b_logit_pred)

            b_logit_pred = b_logit_pred.detach().cpu().numpy()
            pred_label = pred_label.to('cpu').numpy()
            b_labels = b_labels.to('cpu').numpy()

        tokenized_texts.append(b_input_ids)
        logit_preds.append(b_logit_pred)
        true_labels.append(b_labels)
        pred_labels.append(pred_label)

    # Flatten outputs
    pred_labels = [item for sublist in pred_labels for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    # Calculate Accuracy
    threshold = 0.50
    pred_bools = [pl > threshold for pl in pred_labels]
    true_bools = [tl == 1 for tl in true_labels]
    print(true_bools)
    print(pred_bools)
    #val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
    #val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  #print('F1 Validation Accuracy: ', val_f1_accuracy)
  #print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train loss: 0.6501819491386414
Train loss: 0.6806412935256958
Train loss: 0.692043681939443
Train loss: 0.6987106502056122
Train loss: 0.6867193937301636
Train loss: 0.6954268217086792
Train loss: 0.6897677523749215
Train loss: 0.6909393519163132
Train loss: 0.688995467291938
Train loss: 0.6890218198299408
Train loss: 0.6899993473833258
Train loss: 0.6927060981591543
Train loss: 0.6912597326131967
Train loss: 0.6926233768463135
Train loss: 0.692995572090149
Train loss: 0.6919585764408112
Train loss: 0.6881395753692178
Train loss: 0.6883550021383498
Train loss: 0.6867475509643555
Train loss: 0.6880679816007614
Train loss: 0.6893828738303411
Train loss: 0.6911164359612898
Train loss: 0.6910263455432394
Train loss: 0.6913411517937978
Train loss: 0.6918106436729431
Train loss: 0.6917380186227652
Train loss: 0.6911037740884004
Train loss: 0.6909978943211692
Train loss: 0.6897773722122456
Train loss: 0.6887851496537526
Train loss: 0.6878763302679984
Train loss: 0.6877161078155041
Train loss:

Epoch:  33%|███▎      | 1/3 [02:24<04:48, 144.25s/it]

[False, True, False, False, True, True, True, True, False, True, True, False, False, False, True, True, True, False, True, False, False, True, False, False, True, False, True, True, True, False, False, True, False, False, False, True, True, False, False, False, False, False, True, False, True, False, True, True, True, True, False, False, False, True, False, False, True, False, False, True, True, True, False, True, True, True, False, False, False, False, True, True, True, True, True, True, False, False, False, False, True, False, True, False, True, False, False, True, False, True, False, False, True, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, True, False, False, True, True, True, True, True, False, True, False, False, True, True, True, False, False, False, True, False, False, False, False, True, False, False, True, True, False, False, False, True, True, False, False, True, False, False, True, False, False, False, True, False, Tr

Epoch:  67%|██████▋   | 2/3 [04:56<02:28, 148.90s/it]

[False, True, False, False, True, True, True, True, False, True, True, False, False, False, True, True, True, False, True, False, False, True, False, False, True, False, True, True, True, False, False, True, False, False, False, True, True, False, False, False, False, False, True, False, True, False, True, True, True, True, False, False, False, True, False, False, True, False, False, True, True, True, False, True, True, True, False, False, False, False, True, True, True, True, True, True, False, False, False, False, True, False, True, False, True, False, False, True, False, True, False, False, True, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, True, False, False, True, True, True, True, True, False, True, False, False, True, True, True, False, False, False, True, False, False, False, False, True, False, False, True, True, False, False, False, True, True, False, False, True, False, False, True, False, False, False, True, False, Tr

Epoch: 100%|██████████| 3/3 [07:28<00:00, 149.57s/it]

[False, True, False, False, True, True, True, True, False, True, True, False, False, False, True, True, True, False, True, False, False, True, False, False, True, False, True, True, True, False, False, True, False, False, False, True, True, False, False, False, False, False, True, False, True, False, True, True, True, True, False, False, False, True, False, False, True, False, False, True, True, True, False, True, True, True, False, False, False, False, True, True, True, True, True, True, False, False, False, False, True, False, True, False, True, False, False, True, False, True, False, False, True, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, True, False, False, True, True, True, True, True, False, True, False, False, True, True, True, False, False, False, True, False, False, False, False, True, False, False, True, True, False, False, False, True, True, False, False, True, False, False, True, False, False, False, True, False, Tr

In [18]:
sample_text = ["@piamagalona hey ms. P! ur such a FAB mommie!!"]
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
encodings = tokenizer(sample_text, pad_to_max_length = True)
input_ids = torch.tensor(encodings['input_ids'])
attention_masks = torch.tensor(encodings['attention_mask'] )
input_ids

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[    0,  1039,   642,  6009,  1073,   337,  4488, 17232, 43601,     4,
           221,   328, 11540,   215,    10,   274,  4546,  3795, 23738, 12846,
             2]])

In [19]:
outs = model(input_ids.cuda(), attention_mask = attention_masks.cuda())
b_logit_pred = outs[0]
pred_label = torch.sigmoid(b_logit_pred)
print(pred_label)

tensor([[0.1436, 0.8637]], device='cuda:0', grad_fn=<SigmoidBackward0>)


Homework

In [20]:
# Perform Multiclass Classification Using Roberta (You should have minimum of 3 classes)

In [21]:
# dataset: https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/
df_2 = pd.read_csv("/content/drive/My Drive/cyberbullying_tweets.csv", encoding='mac_roman', header = None)
sentiment_classes = list(df_2[1].unique())
df_2[1] = df_2[1].apply(sentiment_classes.index)
df_2 = df_2.sample(10000)
df_2

,0,1
2894,Mama dukes is the best,1
30333,IT IS NOT A BLOCK BOT IT IS A TOOL TO SHARE BL...,4
1715,Success. #stopWadhwa2015,1
13007,RT @A24Brett I'm not sexist but diva matches a...,2
39719,"You can‚Äôt blame them. I mean, given the numb...",5
...,...,...
7682,@edgeofthesandbx @batchelorshow @FearDept Are ...,1
42634,@BomacBrian You a lame ass nigger for talking ...,6
1706,@PeerWorker @EvvyKube it's about ethics in jou...,1
33722,He was chased and bullied by his junior high s...,5


In [22]:
tweets = list(df_2[0])
labels = list(df_2[1])

In [23]:
encodings = tokenizer.batch_encode_plus(tweets, truncation=True, max_length=512, padding='longest')

print('tokenizer outputs: ', encodings.keys())

tokenizer outputs:  dict_keys(['input_ids', 'attention_mask'])


In [24]:
input_ids = encodings['input_ids']
attention_masks = encodings['attention_mask']

In [25]:
tr_inp, val_inp, tr_lbl, val_lbl, tr_mk, val_mk = train_test_split(input_ids, labels, attention_masks, random_state=303, test_size=0.1)

print("x_train", len(tr_inp))
print("x_vad", len(val_inp))
print("y_train", len(tr_lbl))
print("y_vad", len(val_lbl))

x_train 9000
x_vad 1000
y_train 9000
y_vad 1000


In [26]:
tr_inp = torch.tensor(input_ids)
tr_lbl = torch.tensor(labels)
tr_mk = torch.tensor(attention_masks)

val_inp = torch.tensor(val_inp)
val_lbl = torch.tensor(val_lbl)
val_mk = torch.tensor(val_mk)

In [27]:
tr_data = TensorDataset(tr_inp, tr_mk, tr_lbl)
tr_sampler = RandomSampler(tr_data)
tr_dloader = DataLoader(tr_data, sampler=tr_sampler, batch_size=16)

val_data = TensorDataset(val_inp, val_mk, val_lbl)
val_sampler = SequentialSampler(val_data)
val_dloader = DataLoader(val_data, sampler=val_sampler, batch_size=16)

In [28]:
sen_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(sentiment_classes))
sen_model.cuda()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [29]:
param_optimizer = list(sen_model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0
    }
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Train Model
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   2420 MiB |  11561 MiB |  13270 GiB |  13268 GiB |\n|       from large pool |   2404 MiB |  11556 MiB |  13240 GiB |  13238 GiB |\n|       from small pool |     16 MiB |     17 MiB |     30 GiB |     30 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   2420 MiB |  11561 MiB |  13270 GiB |  13268 GiB |\n|       from large pool |   2404 MiB |  11556 MiB |

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loss_set = []

epochs = 3

for _ in trange(epochs, desc="Epoch"):

    # Training
    sen_model.train()

    tr_loss = 0 #running loss
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(tr_dloader):

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()

        outputs = sen_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0].float()
        logits = outputs[1].float()

        loss.backward()

        optimizer.step()

        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

        print("Iteration {} - Train loss: {}".format(nb_tr_steps, tr_loss / nb_tr_steps))

###############################################################################

    # Validation
    sen_model.eval()

    logit_preds, true_labels, pred_labels, tokenized_texts = [],[],[],[]

    # Predict
    for i, batch in enumerate(val_dloader):

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outs = sen_model(b_input_ids, attention_mask=b_input_mask)
            b_logit_pred = outs[0]
            pred_label = torch.softmax(b_logit_pred, dim=1)

            b_logit_pred = b_logit_pred.detach().cpu().numpy()
            pred_label = pred_label.to('cpu').numpy()
            b_labels = b_labels.to('cpu').numpy()

        tokenized_texts.append(b_input_ids)
        logit_preds.append(b_logit_pred)
        true_labels.append(b_labels)
        pred_labels.append(pred_label)

    # Flatten outputs
    pred_labels = [np.argmax(item) for sublist in pred_labels for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]
    print(true_labels)
    print(pred_labels)

    # Calculate Accuracy
    # threshold = 0.50
    # pred_bools = [pl > threshold for pl in pred_labels]
    # true_bools = [tl == 1 for tl in true_labels]
    #val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
    #val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  #print('F1 Validation Accuracy: ', val_f1_accuracy)
  #print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration 1 - Train loss: 0.358257532119751
Iteration 2 - Train loss: 0.32740503549575806
Iteration 3 - Train loss: 0.39285608132680255
Iteration 4 - Train loss: 0.348288431763649
Iteration 5 - Train loss: 0.33591177463531496
Iteration 6 - Train loss: 0.3368089447418849
Iteration 7 - Train loss: 0.3127781025000981
Iteration 8 - Train loss: 0.33049315959215164
Iteration 9 - Train loss: 0.34827351570129395
Iteration 10 - Train loss: 0.33415647149086
Iteration 11 - Train loss: 0.32591859183528205
Iteration 12 - Train loss: 0.30937038734555244
Iteration 13 - Train loss: 0.31716974767354816
Iteration 14 - Train loss: 0.3221028670668602
Iteration 15 - Train loss: 0.3464658945798874
Iteration 16 - Train loss: 0.344036647118628
Iteration 17 - Train loss: 0.3472887584391762
Iteration 18 - Train loss: 0.35431865768300164
Iteration 19 - Train loss: 0.37684538725175354
Iteration 20 - Train loss: 0.381226959079504
Iteration 21 - Train loss: 0.37194051487105234
Iteration 22 - Train loss: 0.378644676

Epoch:  33%|███▎      | 1/3 [14:49<29:39, 889.72s/it]

[6, 6, 5, 5, 3, 3, 3, 6, 2, 2, 5, 6, 2, 3, 5, 2, 2, 2, 6, 1, 1, 5, 1, 2, 4, 6, 5, 6, 6, 3, 1, 3, 5, 4, 4, 1, 3, 2, 3, 4, 6, 3, 3, 6, 5, 1, 3, 6, 6, 4, 3, 2, 6, 2, 6, 6, 1, 4, 5, 3, 2, 3, 4, 6, 6, 5, 4, 4, 3, 3, 1, 2, 5, 1, 4, 3, 1, 4, 6, 2, 3, 5, 1, 4, 1, 4, 3, 2, 2, 6, 6, 2, 6, 4, 6, 3, 3, 2, 4, 1, 5, 5, 4, 6, 5, 1, 5, 1, 1, 4, 4, 6, 3, 4, 2, 6, 3, 2, 1, 5, 6, 5, 5, 2, 5, 3, 2, 5, 5, 2, 2, 5, 5, 6, 6, 5, 1, 3, 1, 4, 6, 4, 6, 3, 4, 3, 6, 2, 2, 2, 6, 5, 6, 6, 1, 1, 3, 1, 4, 2, 6, 2, 1, 3, 2, 6, 3, 2, 5, 6, 1, 2, 3, 5, 2, 4, 4, 6, 6, 3, 2, 1, 3, 3, 2, 5, 4, 3, 5, 5, 3, 1, 4, 2, 5, 5, 4, 4, 4, 6, 3, 5, 6, 2, 3, 2, 5, 6, 1, 6, 3, 4, 6, 5, 3, 6, 4, 6, 6, 2, 4, 1, 3, 1, 3, 1, 1, 6, 4, 6, 3, 2, 2, 5, 5, 5, 4, 3, 6, 1, 5, 1, 1, 4, 3, 4, 2, 3, 6, 3, 3, 4, 4, 3, 6, 5, 5, 4, 1, 3, 2, 3, 5, 1, 2, 6, 6, 4, 6, 2, 2, 3, 4, 6, 3, 6, 3, 5, 5, 3, 2, 3, 1, 6, 5, 6, 6, 5, 2, 2, 4, 3, 5, 2, 3, 1, 2, 6, 5, 4, 4, 2, 6, 4, 3, 2, 6, 6, 3, 3, 5, 6, 2, 2, 6, 6, 5, 3, 4, 1, 1, 1, 5, 6, 2, 5, 1, 2, 5, 4, 4, 5, 4, 

Epoch:  67%|██████▋   | 2/3 [29:43<14:51, 891.85s/it]

[6, 6, 5, 5, 3, 3, 3, 6, 2, 2, 5, 6, 2, 3, 5, 2, 2, 2, 6, 1, 1, 5, 1, 2, 4, 6, 5, 6, 6, 3, 1, 3, 5, 4, 4, 1, 3, 2, 3, 4, 6, 3, 3, 6, 5, 1, 3, 6, 6, 4, 3, 2, 6, 2, 6, 6, 1, 4, 5, 3, 2, 3, 4, 6, 6, 5, 4, 4, 3, 3, 1, 2, 5, 1, 4, 3, 1, 4, 6, 2, 3, 5, 1, 4, 1, 4, 3, 2, 2, 6, 6, 2, 6, 4, 6, 3, 3, 2, 4, 1, 5, 5, 4, 6, 5, 1, 5, 1, 1, 4, 4, 6, 3, 4, 2, 6, 3, 2, 1, 5, 6, 5, 5, 2, 5, 3, 2, 5, 5, 2, 2, 5, 5, 6, 6, 5, 1, 3, 1, 4, 6, 4, 6, 3, 4, 3, 6, 2, 2, 2, 6, 5, 6, 6, 1, 1, 3, 1, 4, 2, 6, 2, 1, 3, 2, 6, 3, 2, 5, 6, 1, 2, 3, 5, 2, 4, 4, 6, 6, 3, 2, 1, 3, 3, 2, 5, 4, 3, 5, 5, 3, 1, 4, 2, 5, 5, 4, 4, 4, 6, 3, 5, 6, 2, 3, 2, 5, 6, 1, 6, 3, 4, 6, 5, 3, 6, 4, 6, 6, 2, 4, 1, 3, 1, 3, 1, 1, 6, 4, 6, 3, 2, 2, 5, 5, 5, 4, 3, 6, 1, 5, 1, 1, 4, 3, 4, 2, 3, 6, 3, 3, 4, 4, 3, 6, 5, 5, 4, 1, 3, 2, 3, 5, 1, 2, 6, 6, 4, 6, 2, 2, 3, 4, 6, 3, 6, 3, 5, 5, 3, 2, 3, 1, 6, 5, 6, 6, 5, 2, 2, 4, 3, 5, 2, 3, 1, 2, 6, 5, 4, 4, 2, 6, 4, 3, 2, 6, 6, 3, 3, 5, 6, 2, 2, 6, 6, 5, 3, 4, 1, 1, 1, 5, 6, 2, 5, 1, 2, 5, 4, 4, 5, 4, 

Epoch: 100%|██████████| 3/3 [44:36<00:00, 892.32s/it]

[6, 6, 5, 5, 3, 3, 3, 6, 2, 2, 5, 6, 2, 3, 5, 2, 2, 2, 6, 1, 1, 5, 1, 2, 4, 6, 5, 6, 6, 3, 1, 3, 5, 4, 4, 1, 3, 2, 3, 4, 6, 3, 3, 6, 5, 1, 3, 6, 6, 4, 3, 2, 6, 2, 6, 6, 1, 4, 5, 3, 2, 3, 4, 6, 6, 5, 4, 4, 3, 3, 1, 2, 5, 1, 4, 3, 1, 4, 6, 2, 3, 5, 1, 4, 1, 4, 3, 2, 2, 6, 6, 2, 6, 4, 6, 3, 3, 2, 4, 1, 5, 5, 4, 6, 5, 1, 5, 1, 1, 4, 4, 6, 3, 4, 2, 6, 3, 2, 1, 5, 6, 5, 5, 2, 5, 3, 2, 5, 5, 2, 2, 5, 5, 6, 6, 5, 1, 3, 1, 4, 6, 4, 6, 3, 4, 3, 6, 2, 2, 2, 6, 5, 6, 6, 1, 1, 3, 1, 4, 2, 6, 2, 1, 3, 2, 6, 3, 2, 5, 6, 1, 2, 3, 5, 2, 4, 4, 6, 6, 3, 2, 1, 3, 3, 2, 5, 4, 3, 5, 5, 3, 1, 4, 2, 5, 5, 4, 4, 4, 6, 3, 5, 6, 2, 3, 2, 5, 6, 1, 6, 3, 4, 6, 5, 3, 6, 4, 6, 6, 2, 4, 1, 3, 1, 3, 1, 1, 6, 4, 6, 3, 2, 2, 5, 5, 5, 4, 3, 6, 1, 5, 1, 1, 4, 3, 4, 2, 3, 6, 3, 3, 4, 4, 3, 6, 5, 5, 4, 1, 3, 2, 3, 5, 1, 2, 6, 6, 4, 6, 2, 2, 3, 4, 6, 3, 6, 3, 5, 5, 3, 2, 3, 1, 6, 5, 6, 6, 5, 2, 2, 4, 3, 5, 2, 3, 1, 2, 6, 5, 4, 4, 2, 6, 4, 3, 2, 6, 6, 3, 3, 5, 6, 2, 2, 6, 6, 5, 3, 4, 1, 1, 1, 5, 6, 2, 5, 1, 2, 5, 4, 4, 5, 4, 

In [41]:
from sklearn.metrics import accuracy_score

# Using the validation dataset to calculate accuracy score
sen_model.eval()

logit_preds, true_labels, pred_labels, tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(val_dloader):

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
        outs = sen_model(b_input_ids, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.softmax(b_logit_pred, dim=1)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

pred_labels = [np.argmax(item) for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]

In [45]:
flat_accuracy = accuracy_score(true_labels, pred_labels) * 100
print(f'Flat accuracy: {flat_accuracy}')

Flat accuracy: 94.3
